In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# bert= BertForSequenceClassification.from_pretrained('bert-base-uncased')
# print(bert.embeddings.word_embeddings)

In [ ]:
!pip install bs4

In [ ]:
import numpy as np
import pandas as pd
import glob
import random
from tqdm import tqdm, trange
import time

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


from transformers import BertTokenizer
from transformers import XLNetTokenizer
from transformers import RobertaTokenizer
from transformers import AlbertTokenizer
from transformers import AdamW

from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from transformers import *
import warnings

# from senti_utility import *  
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

torch.cuda.empty_cache()

In [ ]:
# Hyperparameters
MAX_LEN=256
BATCH_SIZE=16
EPOCHS=4
LEARNING_RATE=2e-5

In [ ]:
MODELS = [(BertForSequenceClassification,BertTokenizer,'bert-base-cased','bert'),
          (XLNetForSequenceClassification, XLNetTokenizer,'xlnet-base-cased','xlnet'),
          (RobertaForSequenceClassification, RobertaTokenizer,'roberta-base','Roberta'), 
          (AlbertForSequenceClassification, AlbertTokenizer,'albert-base-v1','albert')
          ]

MODEL_NAMES = ['bert', 'xlnet', 'Roberta', 'albert']

In [ ]:
def seed_torch(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic=True

seed_torch(42)

In [ ]:
df  = pd.read_csv("../input/sentiment-paper-s4/ResultsConsolidatedWithEnsambleAssessment_BERTLabeled.csv")
df['ManualLabel']=df['ManualLabel'].replace({'o': 0, 'p': 1,'n':2})
final_pred = pd.Series()
dfs = []

In [ ]:
df

In [ ]:
JIRA = df[df['File'].str.contains('LinJIRA')]

In [ ]:
BOGIAS = df[df['File'].str.contains('BenchmarkUddinSO')]
LINAPP = df[df['File'].str.contains('DatasetLinAppReviews')]
LINSO = df[df['File'].str.contains('DatasetLinSO')]
SENTI4S = df[df['File'].str.contains('DatasetSenti4SDSO')]
ORTUJIRA = df[df['File'].str.contains('OrtuJIRA')]

In [ ]:
df = LINAPP
# df = ORTUJIRA

In [ ]:
import re
import pandas as pd 
import nltk
from nltk.stem.snowball import SnowballStemmer
from imblearn.over_sampling import SMOTE
from statistics import mean
import numpy as np
import argparse
import csv
import scipy as sp
from scipy.sparse import coo_matrix, hstack
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.stem.snowball import SnowballStemmer
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
import math
from nltk.tokenize import sent_tokenize, word_tokenize, WordPunctTokenizer
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from multiprocessing import Process

stopWords = set(stopwords.words('english'))
stemmer =SnowballStemmer("english")


mystop_words=[
'i', 'me', 'my', 'myself', 'we', 'our',  'ourselves', 'you', 'your',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her',
'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'themselves',
 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
'and',  'if', 'or', 'as', 'until',  'of', 'at', 'by',  'between', 'into',
'through', 'during', 'to', 'from', 'in', 'out', 'on', 'off', 'then', 'once', 'here',
 'there',  'all', 'any', 'both', 'each', 'few', 'more',
 'other', 'some', 'such',  'than', 'too', 'very', 's', 't', 'can', 'will',  'don', 'should', 'now'
# keywords
 'while', 'case', 'switch','def', 'abstract','byte','continue','native','private','synchronized',
 'if', 'do', 'include', 'each', 'than', 'finally', 'class', 'double', 'float', 'int','else','instanceof',
 'long', 'super', 'import', 'short', 'default', 'catch', 'try', 'new', 'final', 'extends', 'implements',
 'public', 'protected', 'static', 'this', 'return', 'char', 'const', 'break', 'boolean', 'bool', 'package',
 'byte', 'assert', 'raise', 'global', 'with', 'or', 'yield', 'in', 'out', 'except', 'and', 'enum', 'signed',
 'void', 'virtual', 'union', 'goto', 'var', 'function', 'require', 'print', 'echo', 'foreach', 'elseif', 'namespace',
 'delegate', 'event', 'override', 'struct', 'readonly', 'explicit', 'interface', 'get', 'set','elif','for',
 'throw','throws','lambda','endfor','endforeach','endif','endwhile','clone',
 u"'d", u"'s", u'abov', u'ani', u'becaus', u'befor', u'continu', u'could', u'deleg', u'doe', u'doubl', u'dure', 
 u'els', u'endwhil', u'extend', u'implement', u'includ', u'interfac', u'might', u'must', u"n't", u'namespac', 
 u'nativ', u'need', u'nowwhil', u'onc', u'onli', u'ourselv', u'overrid', u'packag', u'privat', u'protect', u'rais', 
 u'readon', u'requir', u'sha', u'sign', u'synchron', u'themselv', u'tri', u'veri', u'whi', u'wo', u'would', u'yourselv',
 u'el', u'rai'
]

infileContractions = "../input/sentiment-auxilary/Contractions.txt"
infileEmotionLookup = "../input/sentiment-auxilary/EmoticonLookupTable.txt"

emodict=[]
contractions_dict=[]
# Read in the words with sentiment from the dictionary
with open(infileContractions,"r") as contractions,\
     open(infileEmotionLookup,"r") as emotable:
    contractions_reader=csv.reader(contractions, delimiter='\t')
    emoticon_reader=csv.reader(emotable,delimiter='\t')

    #Hash words from dictionary with their values
    contractions_dict = {rows[0]:rows[1] for rows in contractions_reader}
    emodict={rows[0]:rows[1] for rows in emoticon_reader}

    contractions.close()
    emotable.close()



grammar= r"""
NegP: {<VERB>?<ADV>+<VERB|ADJ>?<PRT|ADV><VERB>}
{<VERB>?<ADV>+<VERB|ADJ>*<ADP|DET>?<ADJ>?<NOUN>?<ADV>?}

"""
chunk_parser = nltk.RegexpParser(grammar)
contractions_regex = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
     def replace(match):
        return contractions_dict[match.group(0)]
     return contractions_regex.sub(replace, s.lower())


url_regex = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

def remove_url(s):
    return url_regex.sub(" ",s)

negation_words =['not', 'never', 'none', 'nobody', 'nowhere', 'neither', 'barely', 'hardly',
                     'nothing', 'rarely', 'seldom', 'despite' ]

emoticon_words=['PositiveSentiment','NegativeSentiment']

def negated(input_words):
    """
    Determine if input contains negation words
    """
    neg_words = []
    neg_words.extend(negation_words)
    for word in neg_words:
        if word in input_words:
            return True
    return False

def prepend_not(word):
    if word in emoticon_words:
        return word
    elif word in negation_words:
        return word
    return "NOT_"+word

def handle_negation(comments):
    sentences = nltk.sent_tokenize(comments)
    modified_st=[]
    for st in sentences:
        allwords = nltk.word_tokenize(st)
#         print(allwords)
        modified_words=[]
        if negated(allwords):
            part_of_speech = nltk.tag.pos_tag(allwords,tagset='universal')
            chunked = chunk_parser.parse(part_of_speech)
            print(chunked)
            #print("---------------------------")
            #print(st)
            for n in chunked:
                if isinstance(n, nltk.tree.Tree):
                    words = [pair[0] for pair in n.leaves()]
                    #print(words)

                    if n.label() == 'NegP' and negated(words):
                        for i, (word, pos) in enumerate(n.leaves()):
                            if (pos=="ADV" or pos=="ADJ" or pos=="VERB") and (word!="not"):
                                modified_words.append(prepend_not(word))
                            else:
                                modified_words.append(word)
                    else:
                         modified_words.extend(words)
                else:
                    modified_words.append(n[0])
            newst =' '.join(modified_words)
            #print(newst)
            modified_st.append(newst)
        else:
            modified_st.append(st)
    return ". ".join(modified_st)

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')
tok = WordPunctTokenizer()


for w in mystop_words:
    stopWords.add(w)
stopWords = list(stopWords)

def stem_tokens(tokens):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize_and_stem(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens)
    #stems = tokens
    return stems

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def preprocess_text(text):
    if text is None: return []
    #comments = text.encode('ascii', 'ignore')
    text = text.lower()
    text = text.replace("\\", "")
    comments = text
    comments = expand_contractions(comments)
    comments = remove_url(comments)
#     print(emodict)
    comments = replace_all(comments, emodict)
    #comments = tweet_cleaner_updated(comments)
    #comments = Utils.handle_negation(comments)
    #comments = detectSentimentDominentClause(comments)    return  comments
    return comments

In [ ]:
df_prediction={}
for j in range(1,2):
    
    m_num=j
    cur_model=MODELS[m_num]
    print(cur_model[3])
    m_name=MODEL_NAMES[m_num]
    df_model = pd.DataFrame()
    
    df_prediction[m_name] = []
    cross_fold = 10
    for k in range(cross_fold):
        test_df  = df[df['File'].str.contains("_{}".format(k))]
        train_df  = df[~df['File'].str.contains("_{}".format(k))]
#         print(df, test_df)
        tokenizer = cur_model[1].from_pretrained(cur_model[2], do_lower_case=True)
        old_tokenizer_size = len(tokenizer)
        new_token = ['senti4s_o', 'senti4s_n','senti4s_p','senticr_o','senticr_p','senticr_n','sentistrength_o', 'sentistrength_n', 'sentistrength_p','pome_o','pome_n','pome_p','dsolabel_o','dsolabel_n','dsolabel_p']
        tokenizer.add_tokens(new_token)
        sentences=train_df.Sentence.values
        labels=train_df.ManualLabel.values
        senti=train_df.Senti4SD.values
        senticr=train_df.SentiCR.values
        sentistr=train_df.SentistrengthSE.values
        pome=train_df.POME.values
        dso = train_df.DsoLabelFullText.values
#         numericuls = train_df[['Senti4SD','SentiCR','SentistrengthSE','POME','DsoLabelFullText', 'BERT', 'ROBERTA', 'XLNET','ALBERT']].to_numpy()
        
        input_ids = []
        attention_masks = []
        input_numerciculs=[]
#         enc = OneHotEncoder(handle_unknown='ignore')
#         enc.fit(numericuls)
#         for sent,num in zip(sentences,numericuls):
        for sent,l1, l2, l3, l4, l5 in zip(sentences, senti, senticr, sentistr, pome, dso):
            sent = str(sent)
            if len(sent)==0:
                sent = "null"
            #print(sent)
            sent = sent+ ".senti4s_{}.senticr_{}.sentistrength_{}.pome_{}.dsolabel_{}.".format(l1,l2,l3,l4,l5)
            psent = preprocess_text(sent)
#             print(psent)
            encoded_dict = tokenizer.encode_plus(
                                str(psent), 
                                add_special_tokens = True, 
                                max_length = MAX_LEN,
                                pad_to_max_length = True,
                                return_attention_mask = True, 
                                return_tensors = 'pt',
                                truncation=True
                            )
#             print(encoded_dict)
#             break
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])
#             num = np.array(num).reshape(1,9)
#             input_numerciculs.append(torch.from_numpy(enc.transform(num).toarray()))
        train_inputs = torch.cat(input_ids, dim=0)
        train_masks = torch.cat(attention_masks, dim=0)
        train_labels = torch.tensor(labels)
#         train_numericul = torch.cat(input_numerciculs, dim=0)
        
        print('Training data {} {} {}'.format(train_inputs.shape, train_masks.shape, train_labels.shape))
        
        
#         train_data = TensorDataset(train_inputs, train_masks, train_labels,train_numericul)
        train_data = TensorDataset(train_inputs, train_masks, train_labels)
        train_sampler = RandomSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)
        
        # Train Model
#         cur_model[0].num_size = train_numericul.shape[1]
        model = cur_model[0].from_pretrained(cur_model[2], num_labels=3)
        
        model.resize_token_embeddings(len(tokenizer)) 
#         print(model.word_embeddings)
#         for idx in range(len(new_token)):
#             model.embeddings.word_embeddings.weight[old_tokenizer_size+idx, :] = torch.zeros([model.config.hidden_size])
        
        model.cuda()
        
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'gamma', 'beta']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
              'weight_decay_rate': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
              'weight_decay_rate': 0.0}
        ]
        
        optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE)
        
        begin=time.time()
        train_loss_set = []
        
        for _ in trange(EPOCHS, desc="Epoch"): 
            model.train()
        
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            
            for step, batch in enumerate(train_dataloader):
            
                batch = tuple(t.to(device) for t in batch)
              
#                 b_input_ids, b_input_mask, b_labels, b_input_numericul = batch
                b_input_ids, b_input_mask, b_labels = batch
                optimizer.zero_grad()
                # Forward pass
                outputs = model(b_input_ids, token_type_ids=None, \
                                attention_mask=b_input_mask, labels=b_labels ) #b_input_ids, b_input_mask, b_labels, b_input_numericul = batch
                loss = outputs[0]
                logits = outputs[1]
                train_loss_set.append(loss.item())    
                
                # Backward pass
                loss.backward()
                optimizer.step()
                
                tr_loss += loss.item()
                nb_tr_examples += b_input_ids.size(0)
                nb_tr_steps += 1
        
            print("Train loss: {}".format(tr_loss/nb_tr_steps))
        
        end=time.time()
        print('Training used {} second'.format(end-begin))
        
        # Test Model
        begin=time.time()    
        sentences=test_df.Sentence.values
        labels = test_df.ManualLabel.values
#         numericuls = test_df[['Senti4SD','SentiCR','SentistrengthSE','POME','DsoLabelFullText', 'BERT', 'ROBERTA', 'XLNET','ALBERT']].to_numpy()
        input_ids = []
        attention_masks = []
        senti=test_df.Senti4SD.values
        senticr=test_df.SentiCR.values
        sentistr=test_df.SentistrengthSE.values
        pome=test_df.POME.values
        dso = test_df.DsoLabelFullText.values
        
#         input_numerciculs=[]
        
#         for sent ,num in zip(sentences,numericuls):
        for sent, l1, l2,l3,l4,l5 in zip(sentences, senti, senticr, sentistr, pome, dso):
            sent = str(sent)
            if len(sent)==0:
                sent = "null"    
            sent = sent+ ".senti4s_{}.senticr_{}.sentistrength_{}.pome_{}.dsolabel_{}.".format(l1,l2,l3,l4,l5)
            psent = preprocess_text(sent)
            encoded_dict = tokenizer.encode_plus(
                            str(psent), 
                            add_special_tokens = True, 
                            max_length = MAX_LEN,
                            pad_to_max_length = True,
                            return_attention_mask = True, 
                            return_tensors = 'pt'
                            )
             
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])
#             num = np.array(num).reshape(1,9)
#             input_numerciculs.append(torch.from_numpy(enc.transform(num).toarray()))
        prediction_inputs = torch.cat(input_ids,dim=0)
        prediction_masks = torch.cat(attention_masks,dim=0)
        prediction_labels = torch.tensor(labels)
#         prediction_numericul = torch.cat(input_numerciculs, dim=0)
        
        prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels) # ,prediction_numericul
        prediction_sampler = SequentialSampler(prediction_data)
        prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=BATCH_SIZE)
        
        model.eval()
        predictions,true_labels=[],[]
        
        for batch in prediction_dataloader:
            batch = tuple(t.to(device) for t in batch)
#             b_input_ids, b_input_mask, b_labels,b_input_numericul = batch
            b_input_ids, b_input_mask, b_labels = batch
            
        
            with torch.no_grad():
                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask) #,input_numericul=b_input_numericul
                logits = outputs[0]
        
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            
            predictions.append(logits)
            true_labels.append(label_ids)
            
        end=time.time()
        print('Prediction used {:.2f} seconds'.format(end-begin))
        
        flat_predictions = [item for sublist in predictions for item in sublist]
        flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
        flat_true_labels = [item for sublist in true_labels for item in sublist]
        
        print("Accuracy of {} on Jira is: {}".format(m_name, accuracy_score(flat_true_labels,flat_predictions)))
        print(classification_report(flat_true_labels, flat_predictions))
        print(flat_true_labels)
        test_df[cur_model[3]] = flat_predictions
        df_model = df_model.append(test_df)
        torch.cuda.empty_cache()
#         print(len(flat_predictions))
        df_prediction[m_name].extend(flat_predictions)

In [ ]:
# bert = []
# for d in df_prediction['bert']:
#     bert.extend(d)
# roberta = []
# for d in df_prediction['Roberta']:
#     roberta.extend(d)
# xlnet = []
# for d in df_prediction['xlnet']:
#     xlnet.extend(d)
# albert = []
# for d in df_prediction['albert']:
#     albert.extend(d)

In [ ]:
# df_bogias = {'bert':bert, 'roberta': roberta, 'xlnet': xlnet, 'albert': albert}

In [ ]:
# import pickle

In [ ]:
# LINAPP['bert-1'] = df_prediction['bert']
# LINAPP['roberta-1'] = df_prediction['Roberta']
# LINAPP['xlnet-1'] = df_prediction['xlnet']
# LINAPP['albert-1'] = df_prediction['albert']

In [ ]:

# LINAPP['bert-1'] = LINAPP['bert-1'].replace({1:'p',2:'n', 0:'o'})
# LINAPP['xlnet-1'] = LINAPP['xlnet-1'].replace({1:'p',2:'n', 0:'o'})
# LINAPP['albert-1'] = LINAPP['albert-1'].replace({1:'p',2:'n', 0:'o'})
# LINAPP['roberta-1'] = LINAPP['roberta-1'].replace({1:'p',2:'n', 0:'o'})
# LINAPP['ManualLabel'] = LINAPP['ManualLabel'].replace({1:'p',2:'n', 0:'o'})

In [ ]:
# LINAPP.to_csv('test.csv')

In [ ]:
# pickle.dump(df_bogias, open('linapp_run3.p','wb'))